In [ ]:
# note - you will need to be able to run the UNIX ffmpeg utility to run this notebook

In [ ]:
#pip install google-cloud-storage
#!pip install google-cloud-language
#!pip install google-cloud-speech

In [ ]:
import urllib.request
import os
import glob
from google.cloud import storage
from google.cloud import speech
from google.protobuf.json_format import MessageToDict
import yaml
import json
import time

In [ ]:
with open('properties.yaml') as file:
    properties = yaml.full_load(file)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = properties['google_application_credentials']

In [ ]:
bucket_name = properties['bucket_name']
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

In [ ]:
url = "https://ia600702.us.archive.org/22/items/tobacco_dwu03f00/170500121_512kb.mp4"

In [ ]:
source_file_name = url.split('/')[-1]
source_file_stem = source_file_name.split('.')[0]

In [ ]:
start_time = time.time()    
print("downloading file")
urllib.request.urlretrieve(url, 'video_files/' + source_file_name)
print("run time:", time.time() - start_time)

In [ ]:
start_time = time.time()
blob = bucket.blob("video_files/" + source_file_name)
blob.upload_from_filename('video_files/' + source_file_name)

In [ ]:
# convert locally to flac format
print("converting to flac")
os.system('ffmpeg -v quiet -i video_files/' + source_file_name  + ' -c:a flac flac_files/' + source_file_stem + '.flac')
print("run time:", time.time() - start_time)
start_time = time.time()    

In [ ]:
print("uploading flac file to cloud")
blob = bucket.blob(source_file_name)
blob = bucket.blob("flac_files/" + source_file_stem + '.flac')

In [ ]:
# upload
blob.upload_from_filename('flac_files/' + source_file_stem + '.flac')
print("run time:", time.time() - start_time)
start_time = time.time()
# extract transcript

In [ ]:
print("extracting transcript")
client = speech.SpeechClient()

gcs_uri = "gs://" + bucket_name  + "/flac_files/" + source_file_stem + ".flac"
#https://cloud.google.com/speech-to-text/docs/encoding    
#You are not required to specify the encoding and sample rate for WAV or FLAC files. 
#If omitted, Speech-to-Text automatically determines the encoding and sample rate for 
#WAV or FLAC files based on the file header. 
#If you specify an encoding or sample rate value that does not match the value in the 
#file header, then Speech-to-Text returns an error.    
# model='video' is not required, costs more, but might lead to better transcription

audio = speech.RecognitionAudio(uri=gcs_uri)

config = speech.RecognitionConfig(
    #encoding=speech.RecognitionConfig.AudioEncoding.FLAC,
    #sample_rate_hertz=16000,
    audio_channel_count=2,
    language_code="en-US",
    use_enhanced=True,
    model='video',
    enable_word_time_offsets=True
)

operation = client.long_running_recognize(config=config, audio=audio)
response = operation.result()
print("run time:", time.time() - start_time)
start_time = time.time()

In [ ]:
result_dict = MessageToDict(response.__class__.pb(response))
print(result_dict)

In [ ]:
print(result_dict['results'][0]['alternatives'][0]['transcript'])

In [ ]:
bucket.blob("video_files/" + source_file_name).delete()
bucket.blob("flac_files/" + source_file_stem + '.flac').delete()

In [ ]:
os.remove("video_files/" + source_file_name)
os.remove("flac_files/" + source_file_stem + '.flac')